 # **Google Place API -- Boston Restaurants Database**



# import library

In [ ]:
import googlemaps
import pprint
import time
import pandas as pd

# Scraping 20 restaurants each time

In [ ]:
def add_20_restaurants(place_result, boston_restaurants):
  for rest in place_result['results']: 
    permanently_closed = False 
    if 'permanently_closed'in rest:
      permanently_closed = True
    boston_restaurants.append([  rest['name'], 
                    rest['geometry']['location']['lat'],
                    rest['geometry']['location']['lng'],
                    rest['vicinity'],
                    rest['business_status'],
                    permanently_closed,
                    rest['rating'],
                    rest['user_ratings_total'],
                    rest['scope'],
                    ",".join(rest['types'])
                      ]  )
    
  return place_result['next_page_token'], boston_restaurants

# Generate 100 location points in boston 
Because google place API only return 60 result each time.

In [ ]:
# square boston area  due to 60 result ecah return in google map policy
east_location = -71.005180
west_location = -71.244204
north_location = 42.402418
south_location = 42.274505

long = (west_location-east_location)/9
wide = (north_location -south_location )/9
lng_dic = [long*i+east_location   for i in range(10)]
lat_dic = [wide*i+south_location   for i in range(10)]

# 100 location points
locations = [] 
for lat in lat_dic:
  for lng in lng_dic:
    string = str(lat)+","+str(lng)
    locations.append(string) 


# Download restaurants infomation 

In [ ]:
api_key = '************************************'
gmaps = googlemaps.Client(key = api_key)

# using columns in place_result
field = ['name', 'lat', 'lng', 'vicinity','business_status','permanently_closed ', 'rating ','user_ratings_total ', 'source', 'type']


# I try radius 100, 1000, 2000(best), 4000, 100000 
boston_restaurants = []
for location in locations:
  place_result = gmaps.places_nearby(location, radius = 2000 , type = 'restaurant')
  iter = 3
  for i in range(iter):
    try:
      next_page_token ,boston_restaurants = add_20_restaurants(place_result, boston_restaurants)
      time.sleep(3)
      place_result = gmaps.places_nearby(page_token= next_page_token)
    except:
      break
  # pprint.pprint(place_result)

len(boston_restaurants)

3731

# Reduce Repeat Restaurants

In [ ]:
no_repeat_rest = []
for i in boston_restaurants:
  if i not in no_repeat_rest:
    no_repeat_rest.append(i)
print('There are ', str(len(no_repeat_rest)), 'restaurants in boston from google API' )
df = pd.DataFrame(no_repeat_rest, columns = field)
df.head()

There are  1636 restaurants in boston from google API


,name,lat,lng,vicinity,business_status,permanently_closed,rating,user_ratings_total,source,type
0,Tony's Clam Shop,42.275225,-71.005996,"861 Quincy Shore Drive, Quincy",CLOSED_TEMPORARILY,True,4.3,1609,GOOGLE,"restaurant,food,point_of_interest,establishment"
1,Great Chow,42.267550,-71.016865,"17 Beale Street, Quincy",OPERATIONAL,False,3.9,147,GOOGLE,"meal_delivery,restaurant,food,point_of_interes..."
2,La Paloma Mexican Restaurant,42.269169,-71.024356,"195 Newport Avenue, Quincy",OPERATIONAL,False,4.2,545,GOOGLE,"restaurant,food,point_of_interest,establishment"
3,Chipotle Mexican Grill,42.272584,-71.027265,"60 Newport Avenue, Quincy",OPERATIONAL,False,3.9,627,GOOGLE,"restaurant,food,point_of_interest,establishment"
4,China Jade,42.269307,-71.024366,"195 Newport Avenue, Quincy",OPERATIONAL,False,3.5,42,GOOGLE,"meal_delivery,restaurant,food,point_of_interes..."


# Save as file

In [ ]:
df.to_csv('boston_restaurants.csv', index= False)